In [ ]:
import os
files=os.listdir("/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train")
print(files)

In [ ]:
files=sorted(files)
print(files)

In [ ]:
# Imports for Deep Learning
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

# ensure consistency across runs
import numpy as np
import tensorflow as tf


# Imports to view data
import cv2
from glob import glob
from matplotlib import pyplot as plt
from numpy import floor
import random


In [ ]:
data_dir = "/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
target_size = (64, 64)
target_dims = (64, 64, 3) # add channel for RGB
n_classes = 29
val_frac = 0.1
batch_size = 64

data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True, 
                                    validation_split=val_frac)

train_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator   = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, subset="validation")

In [ ]:
my_model = Sequential()
my_model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=target_dims))
my_model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
my_model.add(Flatten())
my_model.add(Dropout(0.5))
my_model.add(Dense(512, activation='relu'))
my_model.add(Dense(n_classes, activation='softmax'))

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
print(val_generator.class_indices)

In [ ]:
my_model.fit_generator(train_generator, epochs=10, validation_data=val_generator)

In [ ]:
my_model.save('gpu_model_epoch=10.model')

In [ ]:
my_model.save('gpu_model_epoch=10.h5')

# **Model testing (Run only Below Code)**

In [ ]:
from keras.models import load_model
from keras.models import load_model
from keras.preprocessing import image
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
my_model= load_model('/kaggle/input/mobile1/gpu_model_epoch10.h5', compile=False)

In [ ]:
test_image= image.load_img('/kaggle/input/mobile/C.jpeg',target_size = (64, 64))
test_image= image.img_to_array(test_image)
test_image= np.expand_dims(test_image,axis =0)
result=my_model.predict(test_image)

img=cv2.imread('/kaggle/input/mobile/C.jpeg')
plt.imshow(img)

def keras_predict(model, image):
    data = np.asarray( image, dtype="int32" )
    pred_probab = model.predict(data)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

pred_probab, pred_class = keras_predict(my_model,test_image)
#print(pred_class, pred_probab)

CATEGORIES = ["A", "B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","del","nothing","space"]
print(CATEGORIES[pred_class])